### Author: Mingfu Liang
### Date: 2019-03-11
### mingfuliang2020@u.northwestern.edu
### Experiment Environment: Intel(R) Core(TM) i7-7700HQ CPU @ 2.80GHz

In [2]:
import numpy as np
from scipy.linalg import null_space

####### if you want to use fraction in calculation, please uncomment the code below ######

#import fractions
#np.set_printoptions(formatter={'all':lambda x: str(fractions.Fraction(x).limit_denominator())})

### Null Space Method for solving the subproblem in active-set quadratic programming algorithm

First I define the null space method function in order to solve the subproblem in active set method to find $p_{k}$

In [3]:
def null_space_method(G,c,A,b):
    '''
    Author: Mingfu Liang
    Date:   2019-03-10
    
    Implementation of the null space method for solving equality constraint quadratic programming
    
    min (1/2)*x.T * G * x + x.T * c
    
    subject to: 
    
            A * x = b

    Input arguments:
    ================
        G: symmetric n * n matrix, as defined in (16.3), which is the coefficient matrix of second order term of x
        c: vector in R^n, which is the coefficient vector of first order term of x
        A: m * n Jacobian of constraints (with m<=n) as defined in (16.3)
        b: vector in R^m whose components are b_i, i in equality constraint set
        
    Output values:
    ==============
        x_sol: the minimizer of the problem 
        lamda_sol: corresponding lagrange multiplier of the problem
    '''
    
    m = A.shape[0]
    n = A.shape[1]
    Z = null_space(A)
    x = np.zeros((n,1))
    h = np.dot(A,x)-b
    g = c + np.dot(G,x)
    #YpY = np.linalg.lstsq(A, -h)[0]
    Y = np.linalg.lstsq(A, np.identity(m),rcond=-1)[0]
    # make sure that [Y|Z] is nonsingular
    while True:
        try:
            a = np.linalg.inv(np.block([Y,Z]))
            break
        except :
            Y = np.random.rand(n,m)
    YpY = np.linalg.lstsq(A, -h)[0]
    pZ = np.linalg.solve(Z.T.dot(G.dot(Z)),-Z.T.dot(G.dot(YpY))-Z.T.dot(g))
    x_sol = YpY+Z.dot(pZ)
    lamda_sol = np.linalg.solve(np.transpose(A.dot(Y)),np.transpose(Y).dot(g+G.dot(x_sol)))
    return x_sol,lamda_sol

### Initialize options for the active-set quadratic programming algorithm

In [4]:
def init_options_optimizeQP():
    '''
   Option initialization

   Author: Mingfu Liang
   Date:   2019-03-10

   Initialize algorithm options for active set method for convex quardratic programming with default values

   Output values:
   ==============

   options:
       This is a structure with field that correspond to algorithmic
       options of our method.  In particular:

       optimizeQP_max_iter:   
           Maximum number of iterations
       optimizeQP_tol:        
           Convergence tolerance for residual
       optimizeQP_output_level:
           Amount of output printed
           0: No output
           1: Only summary information
           2: One line per iteration (good for debugging)
    '''
    
    options = {}
    options['optimizeQP_max_iter'] = 1e5
    options['optimizeQP_tol'] = 1e-6
    options['optimizeQP_output_level'] = 2
    
    
    return options

### The active-set quadratic programming algorithm

I decide to make the optimizeQP more flexible by let the user to send both the index set of all the constraints and the index set of inequality constraints. **These make my active-set method more general and can handle the case with both inequality constraints and equality constraints**.

In [5]:
def optimizeQP(G, c, A, b, x_0, w_0,C,I,options):
    '''
    Author: Mingfu Liang
    Date:   2019-03-10
    

    
    Input arguments:
    ================
    G: 
        symmetric n * n matrix, as defined in (16.3), which is the coefficient matrix of second order term of x
    
    c: 
        vector in R^n, which is the coefficient vector of first order term of x
    
    A: 
        m * n Jacobian of constraints (with m<=n) as defined in (16.3)
    
    b: 
        vector in R^m whose components are b_i, i in equality constraint set
    
    x_0: 
        the starting point of the quadratic problem
    
    w_0: 
        the starting working set
    
    C: 
        all the constraint index list
    
    I: 
        the inequality constraints index list
    
    options:  
        This is a structure with options for the algorithm. 
        For details see the init_options_optimizeQP function. 
              
    Output values:  
    ============== 
    
    x_sol: the optimal primal variables
    
    lambda_sol: the optimal dual variables
    
    w_sol: the final working set
    status: 
            Return code indicating reason for termination: 
            0: Critical point found (convergence tolerance satisfied) 
           -1: Maximum number of iterations exceeded 
    stats: 
            Structure with statistics for the run. Its fields are 
            num_iter   Number of iterations taken 
            norm_p_k   Norm of step p_k at final iterate
            alpha_k    step-length parameter alpha_k at final iterate
            
    % options:
    ==============
            This is a structure with field that correspond to algorithmic 
            options of our method. In particular: 
            
            max_iter:  
                    Maximum number of iterations 
            tol:  
                    Convergence tolerance for active set method 
            output_level:  
                    Amount of output printed 
                    0: No output 
                    1: Only summary information 
                    2: One line per iteration (good for debugging) 

    '''
    max_iter = options['optimizeQP_max_iter']
    tol = options['optimizeQP_tol']
    output_level = options['optimizeQP_output_level']
    
    # initialize return flag (set to -99 so that we do not accidentally
    # declare success.)
    status = -99
    
    # initialize iteration counter
    iter_count = 0
    
    # initialize  zero-th iteration for output of starting working set, starting point x_0,step-length parameter alpha_k
    # step size p_k, function value f_k,infinite norm of p_k, the index of 
    # blocking constraints' and the index of delete_constraints'
    w_k = w_0
    x_k = x_0
    alpha_k = 0.0
    p_k = np.zeros(x_k.shape)
    f_k = 0.5*np.transpose(x_k).dot(G.dot(x_k))+np.transpose(c).dot(x_k)
    norm_p_k = np.linalg.norm(p_k, np.inf)
    blocking_constraint = 0
    delete_constraint = 0
    
    # Print header and zero-th iteration for output
    if output_level >= 2:
        # (This is just a fancy way to create a string.  The '%s' formatting
        # makes it easy to align the header with the actual output.)
        output_header = '%6s %9s %9s %9s %18s %18s %7s' % \
            ('iter', 'f_k','||p_k||', 'alpha_k','add constraint', 'remove constraint', '|w_k|')
        print(output_header)
        print('%6i %10.2e %9.2e %9.2e %18i %18i %7i' % (iter_count, f_k, norm_p_k, alpha_k, blocking_constraint, delete_constraint, len(w_k)))
    

    while 1:
            ##############################################################
            # Check termination tests
            ##############################################################
            if iter_count >= max_iter:
                # Set flag to indicate the maximum number of iterations has been
                # exceeded
                status = -1
                # The following command says that we now want to leave the current
                # loop (which is the while loop here).  The program execution will
                # resume immediately after the end of the loop
                break
            #############################################################
            
            if w_k: #w_k is not empty set
                A_sub = []
                b_sub = []
                for i in range(0,len(w_0)):
                    # -1 since python start from 0 
                    A_sub.append(A[w_k[i]-1,].tolist())

            b_sub = np.zeros((len(w_k),1))
            g_k = G.dot(x_k) + c

            # solve the subproblem and find the p_k
            # if w_k is empty set, solve p_k as unconstraint convex programming
            # if w_k is not empty set, solve p_k using null space method
            if not w_k:
                # w_k is empty set 
                p_k = np.linalg.solve(G,-g_k)

            else:
                # w_k is not empty set
                A_sub = np.array(A_sub)
                
                # solve the subproblem and return p_k and lamda_sol 
                # here I use Null Space Method to return lamda_sol
                # since it is the same as using (16.42) in book to 
                # get lamda_i_hat. Here i is in working set as 
                # define in book 16.42.
                p_k,lamda_sol = null_space_method(G,g_k,A_sub,b_sub)
                
            # define the norm_p_k    
            norm_p_k = np.linalg.norm(p_k, np.inf)
            if norm_p_k<= tol: # p_k = 0
                # first find out the intersection set between Inequality constraint and working set
                w_k_intersect_I_list = list(set([x for x in I] ).intersection(set([x for x in w_k])))
                w_k_intersect_I_index = []
                for i in range(0,len(w_k_intersect_I_list)):
                    w_k_intersect_I_ind = w_k.index(w_k_intersect_I_list[i])
                    w_k_intersect_I_index.append(w_k_intersect_I_ind)
                    
                # define lamda_i_hat for those lamda_i whose i is in the 
                # the intersection set of Inequality constraint index set and working set
                lamda_i_hat = lamda_sol[w_k_intersect_I_index,]
                
                if min(lamda_i_hat) >= -tol:
                    x_sol = x_k
                    status = 0
                    f_k = 0.5*np.transpose(x_k).dot(G.dot(x_k))+np.transpose(c).dot(x_k)
                    iter_count +=1
                    alpha_k = 0
                    ################## print the final output ################################
                    if output_level >= 2:
                    # Print the output header every 10 iterations
                        if iter_count % 10 == 0:
                            output_header = '%6s %9s %9s %9s %18s %18s %7s' % \
                            ('iter', 'f_k','||p_k||', 'alpha_k','add constraint', 'remove constraint', '|w_k|')
                            print(output_header)
                        print('%6i %10.2e %9.2e %9.2e %18i %18i %7i' % \
                              (iter_count, f_k, norm_p_k, alpha_k,blocking_constraint, delete_constraint, len(w_k)))
                    ##########################################################################
                    
                    break
                else:
                # use min so that even here are several multi-pliers that have the same most negative value
                # it can always pick the one with the smallest constraint index
                    j_ind = np.where(lamda_sol == min(lamda_i_hat)[0])[0][0]
                    
                    # delete the constraint with most negative value lamda
                    delete_constraint = w_k[j_ind]
                    del w_k[j_ind]
                    
            else: # p_k != 0
                # compute alpha_k
                # define non working set based on all the constraint index removing the working set index
                non_w_k_list=list(set([x - 1 for x in C] ) - set([x - 1 for x in w_k]))
                
                block_constraint_list = [] # start from zero, be careful to add one later
                alpha_k_list = [] # start from zero

                for i in range(0,len(non_w_k_list)):
                    # evaluate whether the a_i*p_k is negative
                    if A[non_w_k_list[i],].dot(p_k)<0:
                        alpha_k = (b[non_w_k_list[i],]-np.transpose(A[non_w_k_list[i],]).dot(x_k))/A[non_w_k_list[i],].dot(p_k)
                        if alpha_k <1: 
                            # for each i in non working set, when the corresponding alpha_k smaller than 1, 
                            # means that the constraint is blocking, therefore add the index
                            # of this blocking constraint to the blocking constraint list and 
                            # we only store those alpha_k smaller than 1 into the alpha_k_list
                            # so that we can make sure that when we find out the smallest alpha_k
                            # we can always find out the corresponding blocking constraint.
                            # FYI, you can refer to Page 469 in the book
                            block_constraint_list.append(non_w_k_list[i])
                            alpha_k_list.append(alpha_k)
                            
                # check whether block constraint set is empty
                # if it is empty, which means no blocking constraint
                # then we should set alpha_k = 1
                if not block_constraint_list:
                    alpha_k =1

                else:
                    # find out the smallest alpha_k in alpha_k_list, all the alpha_k
                    # in alpha_k_list are smaller than 1.
                    alpha_k = min(alpha_k_list)
                    
                    # use argmin so that even there are several blocking constraints with same smallest alpha_k, 
                    # we can still pick the one with the smallest variable index.
                    blocking_constraint = block_constraint_list[np.argmin(alpha_k_list)] + 1
                    
                    # adding blocking constraint to working set
                    w_k.append(blocking_constraint)
            # update iteration counter and x_k, calculate current value of the objective function
            iter_count +=1
            x_k = x_k + alpha_k*p_k
            f_k = 0.5*np.transpose(x_k).dot(G.dot(x_k))+np.transpose(c).dot(x_k)
            # Iteration output
            if output_level >= 2:
                # Print the output header every 10 iterations
                if iter_count % 10 == 0:
                    output_header = '%6s %9s %9s %9s %18s %18s %7s' % \
                    ('iter', 'f_k','||p_k||', 'alpha_k','add constraint', 'remove constraint', '|w_k|')
                    print(output_header)
                print('%6i %10.2e %9.2e %9.2e %18i %18i %7i' % \
                      (iter_count, f_k, norm_p_k, alpha_k,blocking_constraint, delete_constraint, len(w_k)))
            # reset the index of blocking constraint and delete constraint
            blocking_constraint = 0
            delete_constraint = 0

    
    
    ###########################
    # End of main loop
    ###########################
    f_sol = 0.5*np.transpose(x_sol).dot(G.dot(x_sol))+np.transpose(c).dot(x_sol)
    w_sol = w_k
    lambda_sol = lamda_sol
    ###########################
    # Finalize results
    ###########################

    # Set the statistics
    stats = {}
    stats['num_iter'] = iter_count
    stats['norm_p_k'] = norm_p_k
    stats['alpha_k'] = alpha_k
    
    # Final output message
    if output_level >= 1:
        print('')
        print('Final objective.................: %g' % f_sol)
        print('Final point.....................: \n' + str(x_sol))
        print('Number of iterations............: %d' % iter_count)
        print('')
        if status == 0:
            print('Exit: Critical point found.')
        elif status == -1:
            print('Exit: Maximum number of iterations (%d) exceeded.' %
                  iter_count)
        else:
            print('ERROR: Unknown status value: %d\n' % status)

    # Return output arguments
    return x_sol, lambda_sol, w_sol, status, stats
        
    
    
    